In [1]:
!pip install seaborn
!pip install torch
!pip install torchvision
!pip install scikit-learn

     |████████████████████████████████| 292 kB 14.7 MB/s eta 0:00:01
     |████████████████████████████████| 9.5 MB 30.5 MB/s eta 0:00:01
     |████████████████████████████████| 25.9 MB 145 kB/s  eta 0:00:01
     |████████████████████████████████| 509 kB 34.9 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 881.9 MB 8.9 kB/s  eta 0:00:01    |███▎                            | 90.3 MB 209 kB/s eta 1:03:08��████████▋    | 761.4 MB 1.0 MB/s eta 0:01:59
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 23.3 MB 1.6 MB/s eta 0:00:011
     |████████████████████████████████| 881.9 MB 8.2 kB/s eta 0:00:011
  Attempting uninstall: torch
    Found existing installation: torch 1.10.2
    Uninstalling torch-1.10.2:
      Successfully uninstalled torch-1.10.2
You should consider upgrading via the '/usr/bin/python3 -

In [2]:
!pip install timm

     |████████████████████████████████| 549 kB 17.4 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 1.0 MB/s  eta 0:00:01
     |████████████████████████████████| 677 kB 10.8 MB/s eta 0:00:01
     |████████████████████████████████| 40 kB 1.0 MB/s  eta 0:00:01
     |████████████████████████████████| 78 kB 1.2 MB/s  eta 0:00:01
  Attempting uninstall: packaging
    Found existing installation: packaging 20.8
    Uninstalling packaging-20.8:
      Successfully uninstalled packaging-20.8
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
!pip install scikit-image

     |████████████████████████████████| 12.4 MB 37.2 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 27.7 MB/s eta 0:00:01     |█████████▎                      | 1.3 MB 27.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 17.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 28.9 MB/s eta 0:00:01
     |████████████████████████████████| 148 kB 49.2 MB/s eta 0:00:01
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

imageio 2.15.0 requires pillow>=8.3.2, but you'll have pillow 8.1.0 which is incompatible.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [6]:
!pip install captum

     |████████████████████████████████| 1.3 MB 21.4 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [18]:
!pip install openpyxl

     |████████████████████████████████| 251 kB 12.8 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


     |████████████████████████████████| 95.4 MB 120 kB/s  eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
anceled
ERROR: Operation cancelled by user
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image
import numpy as np
from skimage.segmentation import slic
from scipy.stats import entropy
import os
from matplotlib import cm
import pandas as pd
import matplotlib.pyplot as plt
from skimage.transform import resize
from captum.attr import IntegratedGradients, Saliency, InputXGradient, NoiseTunnel
from skimage.transform import resize
import gc
from openpyxl import Workbook
from tensorflow.keras.preprocessing.image import load_img

MODEL_PATH = "resnet50_multiclass.pth"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
device = "cuda" if torch.cuda.is_available() else "cpu"

DATASET_ROOT = "./face_extraction/faceforensis_face_extract/test/"
class_names = ["DeepFakeDetection", "FaceShifter", "FaceSwap", "NeuraTextures", "deepfakes", "face2face", "original"]
explic_methods = ["Saliency", "InputXGradient", "IntegratedGradients", "SmoothGrad", "GradCAM"]

print("Iniciando.")

# --- CARREGAR O MODELO ---
# Using torchvision.models.efficientnet_b7 (same as training)
# Carrega modelo
# Dispositivo


model = models.resnet50(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 7)  # Ajustar para 7 classes
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model = model.to(DEVICE)
model.eval()
# --- AVALIAÇÃO ---
all_preds = []
all_labels = []



def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # ajuste para ConvNeXt
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path).convert("RGB")
    tensor = transform(image).unsqueeze(0).to(DEVICE)
    np_img = np.array(image.resize((224, 224))) / 255.0
    return tensor, np_img

def make_gradcam(model, img_tensor, target_layer='layer4.2.conv3'):
    gradients, activations = [], []

    def backward_hook(module, grad_input, grad_output):
        gradients.append(grad_output[0])

    def forward_hook(module, input, output):
        activations.append(output)

    target_module = dict(model.named_modules())[target_layer]

    fh = target_module.register_forward_hook(forward_hook)
    bh = target_module.register_full_backward_hook(backward_hook)

    img_tensor.requires_grad_(True)
    output = model(img_tensor)
    class_idx = torch.argmax(output)
    loss = output[0, class_idx]
    model.zero_grad()
    loss.backward()

    grad = gradients[0]
    act = activations[0]
    pooled_grad = torch.mean(grad, dim=[0, 2, 3])
    for i in range(act.shape[1]):
        act[:, i, :, :] *= pooled_grad[i]
    heatmap = act.mean(dim=1).squeeze().detach().cpu().numpy()
    heatmap = np.maximum(heatmap, 0)
    heatmap = heatmap / (heatmap.max() + 1e-12)

    fh.remove()
    bh.remove()
    return heatmap


def get_saliency_map(method, model, input_tensor, target_idx):
    input_tensor = input_tensor.clone().detach().requires_grad_(True)

    if method == "Saliency":
        explainer = Saliency(model)
        attributions = explainer.attribute(input_tensor, target=target_idx)
    elif method == "InputXGradient":
        explainer = InputXGradient(model)
        attributions = explainer.attribute(input_tensor, target=target_idx)
    elif method == "IntegratedGradients":
        explainer = IntegratedGradients(model)
        attributions = explainer.attribute(input_tensor, target=target_idx, n_steps=10, internal_batch_size=5)
    elif method == "SmoothGrad":
        saliency = Saliency(model)
        explainer = NoiseTunnel(saliency)
        attributions = explainer.attribute(input_tensor, nt_type='smoothgrad', nt_samples=15, target=target_idx)
    elif method == "GradCAM":
        cam = make_gradcam(model, input_tensor)
        cam_resized = resize(cam, (224, 224), mode='reflect', anti_aliasing=True)
        return cam_resized
    else:
        raise ValueError(f"Método {method} desconhecido.")

    attr_np = attributions.detach().cpu().numpy().squeeze()
    attr_np = np.abs(attr_np).sum(axis=0)
    attr_np = (attr_np - attr_np.min()) / (attr_np.max() - attr_np.min() + 1e-12)
    return attr_np

def salience_entropy_old(smap):
    smap = smap / (np.sum(smap) + 1e-12)
    return entropy(smap.flatten() + 1e-12)

def salience_entropy(smap, p=8):
    """
    Calcula a entropia normalizada de um mapa de saliência conforme a equação do artigo.

    smap: mapa de saliência (2D array)
    p: profundidade de bits (p=8 para mapas 8-bit, p=np.inf para valores reais)
    """
    m, n = smap.shape
    
    # 1. Histograma das intensidades
    if np.isinf(p):
        # Para valores reais, usamos um número de bins proporcional ao tamanho
        num_bins = int(np.ceil(np.log2(m * n)))
    else:
        num_bins = 2 ** p
    
    hist, _ = np.histogram(smap, bins=num_bins, range=(smap.min(), smap.max()), density=False)
    
    # 2. Probabilidade p(x_i)
    p_x = hist / hist.sum()
    p_x = p_x[p_x > 0]  # remover zeros para evitar log(0)
    
    # 3. Entropia de Shannon
    entropy_val = -np.sum(p_x * np.log2(p_x))
    
    # 4. Profundidade efetiva  p̂
    p_hat = min(p, np.log2(m * n))
    
    # 5. Entropia máxima possível (eq. 2 do artigo)
    S_max = (1 / (2 ** p_hat)) * (m * n) * (np.log2(m * n) - 2 * p_hat)
    
    # 6. Normalização
    S_entropy = entropy_val / S_max
    
    return S_entropy



def reaction_to_noise(model, img_tensor, salmap):
    noisy = img_tensor + torch.normal(0, 0.05, img_tensor.shape).to(DEVICE)
    output = model(noisy)
    target_idx = torch.argmax(output).item()
    noisy_salmap = get_saliency_map("GradCAM", model, noisy, target_idx)
    
    # Redimensiona o mapa ruidoso para o tamanho do original
    noisy_salmap_resized = resize(noisy_salmap, salmap.shape, mode='reflect', anti_aliasing=True)
    
    return np.mean(np.abs(salmap - noisy_salmap_resized))

def geometrical_robustness(model, raw_img, salmap):
    flipped = np.flip(raw_img, axis=1).copy()  # flip horizontal
    flipped_tensor = torch.from_numpy(flipped.transpose(2, 0, 1)).float().unsqueeze(0).to(DEVICE)
    flipped_tensor.requires_grad_(True)

    output = model(flipped_tensor)
    target_idx = torch.argmax(output).item()
    flipped_salmap = get_saliency_map("GradCAM", model, flipped_tensor, target_idx)

    # Redimensiona para o shape original do mapa
    flipped_salmap_resized = resize(flipped_salmap, salmap.shape, mode='reflect', anti_aliasing=True)

    return np.mean(np.abs(salmap - flipped_salmap_resized))

def accuracy_over_segmentation(model, image, saliency_map, segments=50):
    salmap_resized = resize(saliency_map, (image.shape[0], image.shape[1]), mode='reflect', anti_aliasing=True)
    img_uint8 = (image * 255).astype(np.uint8)
    segs = slic(img_uint8, n_segments=segments, compactness=20, enforce_connectivity=True, start_label=0)
    relevance = np.array([np.mean(salmap_resized[segs == i]) for i in range(np.max(segs) + 1)])
    top_k = np.argsort(relevance)[-int(segments * 0.2):]
    mask = np.isin(segs, top_k).astype(np.float32)
    if mask.ndim == 2:
        mask = np.repeat(mask[:, :, np.newaxis], 3, axis=2)
    degraded = image * (1 - mask)
    with torch.no_grad():
        orig_pred = model(torch.from_numpy(image.transpose(2, 0, 1)).float().unsqueeze(0).to(DEVICE))[0]
        degr_pred = model(torch.from_numpy(degraded.transpose(2, 0, 1)).float().unsqueeze(0).to(DEVICE))[0]
    return np.abs(orig_pred.cpu().numpy() - degr_pred.cpu().numpy()).mean(), segs

def load_image_tensor(image_path):
    """
    Carrega uma imagem e retorna:
    - img_tensor: tensor normalizado para o modelo (1, 3, 224, 224)
    - img_np: imagem normalizada [0,1] em formato numpy para visualização
    """
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path).convert("RGB")
    img_tensor = transform(image).unsqueeze(0)
    img_np = np.array(image.resize((224, 224))) / 255.0
    return img_tensor, img_np





def overlay_heatmap(image_np, heatmap):
    """
    Sobrepõe um mapa de calor na imagem original sem usar OpenCV.
    Usa colormap 'jet' do Matplotlib.
    """
    # Normaliza o heatmap para [0, 1]
    heatmap_norm = (heatmap - np.min(heatmap)) / (np.max(heatmap) - np.min(heatmap) + 1e-12)
    
    # Converte o heatmap para RGB usando colormap 'jet'
    colormap = cm.get_cmap('jet')
    heatmap_colored = colormap(heatmap_norm)[:, :, :3]  # remove canal alpha
    
    # Combina a imagem original com o heatmap
    overlay = 0.4 * heatmap_colored + 0.6 * image_np
    overlay = np.clip(overlay, 0, 1)
    
    return overlay


def generate_visualizations(model, img_tensor, img_np, target_idx):
    """
    Gera visualizações para todas as técnicas de explicabilidade definidas.
    Retorna uma lista de imagens numpy normalizadas [0,1].
    """
    techniques = ["GradCAM", "Saliency", "InputXGradient", "IntegratedGradients", "SmoothGrad"]
    visualizations = []

    for method in techniques:
        salmap = get_saliency_map(method, model, img_tensor, target_idx)

        # Garantir que o mapa está normalizado
        salmap_norm = (salmap - np.min(salmap)) / (np.max(salmap) - np.min(salmap) + 1e-12)

        # Sobrepor mapa na imagem original
        overlay_img = overlay_heatmap(img_np, salmap_norm)
        visualizations.append(overlay_img)

    return visualizations


print("Modelo carregado com sucesso.")

image_files = []
image_labels = []
for folder in class_names:
    folder_path = os.path.join(DATASET_ROOT, folder)
    files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.png'))])
    for f in files:
        image_files.append(os.path.join(folder_path, f))
        image_labels.append(folder)


resultados = []
n_rows = len(image_files)
n_cols = len(explic_methods) + 1

# fig, axs = plt.subplots(n_rows, n_cols, figsize=(3 * n_cols, 3 * n_rows))
maximo=20
natual=0
for row, (path, label) in enumerate(zip(image_files, image_labels)):
    natual=natual+1
    #if (natual>maximo):
    #    break

    img_tensor, raw_img = preprocess_image(path)
    output = model(img_tensor)
    pred_idx = torch.argmax(output, dim=1).item()

    for col, method in enumerate(explic_methods, start=1):
        salmap = get_saliency_map(method, model, img_tensor, pred_idx)
        entropy_val = salience_entropy(salmap)
        noise_val = reaction_to_noise(model, img_tensor, salmap)
        geo_val = geometrical_robustness(model, raw_img, salmap)
        aos_val, _ = accuracy_over_segmentation(model, raw_img, salmap)

        resultado = {
            "imagem": os.path.basename(path),
            "classe_real": label,
            "classe_predita": class_names[pred_idx],
            "metodo_explicabilidade": method,
            "entropia": entropy_val,
            "ruido": noise_val,
            "geo": geo_val,
            "aos": aos_val
        }
        resultados.append(resultado)

        print(f"[{row+1}/{n_rows}] Imagem: {os.path.basename(path)} | Técnica: {method} | Entropia: {entropy_val:.4f} | Ruído: {noise_val:.4f} | Geo: {geo_val:.4f} | AOS: {aos_val:.4f}")

    del img_tensor, raw_img, output
    
    
    torch.cuda.empty_cache()
    gc.collect()


image_groups = {
    "ORIGINAL": [
        "./face_extraction/faceforensis_face_extract/train/original/original_000_frame5.png",
        "./face_extraction/faceforensis_face_extract/train/original/original_004_frame2.png",
        "./face_extraction/faceforensis_face_extract/train/original/original_006_frame1.png",
        "./face_extraction/faceforensis_face_extract/train/original/original_011_frame4.png",
        "./face_extraction/faceforensis_face_extract/train/original/original_018_frame3.png",
    ],
    "FACE2FACE": [
        "./face_extraction/faceforensis_face_extract/train/face2face/Face2Face_000_003_frame5.png",
        "./face_extraction/faceforensis_face_extract/train/face2face/Face2Face_002_006_frame3.png",
        "./face_extraction/faceforensis_face_extract/train/face2face/Face2Face_006_002_frame5.png",
        "./face_extraction/faceforensis_face_extract/train/face2face/Face2Face_010_005_frame3.png",
        "./face_extraction/faceforensis_face_extract/train/face2face/Face2Face_017_803_frame3.png",
    ],
    "FACESHIFTER": [
        "./face_extraction/faceforensis_face_extract/train/FaceShifter/FaceShifter_000_003_frame5.png",
        "./face_extraction/faceforensis_face_extract/train/FaceShifter/FaceShifter_002_006_frame5.png",
        "./face_extraction/faceforensis_face_extract/train/FaceShifter/FaceShifter_006_002_frame5.png",
        "./face_extraction/faceforensis_face_extract/train/FaceShifter/FaceShifter_010_005_frame3.png",
        "./face_extraction/faceforensis_face_extract/train/FaceShifter/FaceShifter_017_803_frame3.png",
    ],
    "FACESWAP": [
        "./face_extraction/faceforensis_face_extract/train/FaceSwap/FaceSwap_000_003_frame5.png",
        "./face_extraction/faceforensis_face_extract/train/FaceSwap/FaceSwap_002_006_frame5.png",
        "./face_extraction/faceforensis_face_extract/train/FaceSwap/FaceSwap_006_002_frame5.png",
        "./face_extraction/faceforensis_face_extract/train/FaceSwap/FaceSwap_010_005_frame3.png",
        "./face_extraction/faceforensis_face_extract/train/FaceSwap/FaceSwap_017_803_frame3.png",
    ],
    "DEEPFAKEDETECTION": [
        "./face_extraction/faceforensis_face_extract/train/DeepFakeDetection/DeepFakeDetection_01_02__talking_against_wall__YVGY8LOK_frame4.png",
        "./face_extraction/faceforensis_face_extract/train/DeepFakeDetection/DeepFakeDetection_01_04__talking_angry_couch__0XUW13RW_frame3.png",
        "./face_extraction/faceforensis_face_extract/train/DeepFakeDetection/DeepFakeDetection_01_15__kitchen_still__02HILKYO_frame5.png",
        "./face_extraction/faceforensis_face_extract/train/DeepFakeDetection/DeepFakeDetection_01_21__walk_down_hall_angry__03X7CELV_frame1.png",
        "./face_extraction/faceforensis_face_extract/train/DeepFakeDetection/DeepFakeDetection_01_27__meeting_serious__ZYCZ30C0_frame3.png",
    ],
    "DEEPFAKES": [
        "./face_extraction/faceforensis_face_extract/train/deepfakes/Deepfakes_000_003_frame5.png",
        "./face_extraction/faceforensis_face_extract/train/deepfakes/Deepfakes_002_006_frame5.png",
        "./face_extraction/faceforensis_face_extract/train/deepfakes/Deepfakes_006_002_frame5.png",
        "./face_extraction/faceforensis_face_extract/train/deepfakes/Deepfakes_010_005_frame3.png",
        "./face_extraction/faceforensis_face_extract/train/deepfakes/Deepfakes_017_803_frame3.png",
    ],
    "NEURALTEXTURES": [
        "./face_extraction/faceforensis_face_extract/train/NeuraTextures/NeuralTextures_000_003_frame5.png",
        "./face_extraction/faceforensis_face_extract/train/NeuraTextures/NeuralTextures_002_006_frame5.png",
        "./face_extraction/faceforensis_face_extract/train/NeuraTextures/NeuralTextures_006_002_frame5.png",
        "./face_extraction/faceforensis_face_extract/train/NeuraTextures/NeuralTextures_010_005_frame3.png",
        "./face_extraction/faceforensis_face_extract/train/NeuraTextures/NeuralTextures_017_803_frame3.png",
    ],
}    
def show_explanations_grid():
    techniques = ["GradCAM", "Saliency", "InputXGradient", "IntegratedGradients", "SmoothGrad"]
    fig, axs = plt.subplots(len(image_groups), len(techniques), figsize=(5*len(techniques), 4*len(image_groups)))
    for row_idx, (label, image_list) in enumerate(image_groups.items()):
        img_tensor, img_np = load_image_tensor(image_list[0])
        img_tensor = img_tensor.to(device)
        pred = torch.argmax(torch.nn.functional.softmax(model(img_tensor), dim=1)).item()
        vis_list = generate_visualizations(model, img_tensor, img_np, pred)
        for col_idx, vis in enumerate(vis_list):
            axs[row_idx, col_idx].imshow(vis)
            axs[row_idx, col_idx].axis('off')
            if row_idx == 0:
                axs[row_idx, col_idx].set_title(techniques[col_idx], fontsize=12)
            if col_idx == 0:
                axs[row_idx, col_idx].set_ylabel(label, fontsize=12)
                
        del img_tensor, vis_list
        torch.cuda.empty_cache()
    plt.tight_layout()
    plt.savefig("explicabilidade_grid_efficientnet_Resnet.png")
    print("✅ Figura salva como 'explicabilidade_grid_efficientnet_Resnet.png'")
    plt.show(block=True)

if __name__ == "__main__":
    #plt.tight_layout()
    #plt.savefig("grid_explicabilidade_todas_imagens.png", dpi=300)
    #print("Grid salvo como grid_explicabilidade_todas_imagens.png")

    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_excel("resultados_explicabilidade_todas_imagens_resnet.xlsx", index=False)
    print("Planilha salva como resultados_explicabilidade_todas_imagens_resnet.xlsx")
    show_explanations_grid()


Iniciando.
Modelo carregado com sucesso.
[1/6999] Imagem: DeepFakeDetection_01_02__meeting_serious__YVGY8LOK_frame1.png | Técnica: Saliency | Entropia: -0.0529 | Ruído: 0.3793 | Geo: 0.2836 | AOS: 5.6033
[1/6999] Imagem: DeepFakeDetection_01_02__meeting_serious__YVGY8LOK_frame1.png | Técnica: InputXGradient | Entropia: -0.0584 | Ruído: 0.3534 | Geo: 0.2769 | AOS: 8.4320
[1/6999] Imagem: DeepFakeDetection_01_02__meeting_serious__YVGY8LOK_frame1.png | Técnica: IntegratedGradients | Entropia: -0.0643 | Ruído: 0.3560 | Geo: 0.2685 | AOS: 6.4671
[1/6999] Imagem: DeepFakeDetection_01_02__meeting_serious__YVGY8LOK_frame1.png | Técnica: SmoothGrad | Entropia: -0.0622 | Ruído: 0.3279 | Geo: 0.2726 | AOS: 4.1277
[1/6999] Imagem: DeepFakeDetection_01_02__meeting_serious__YVGY8LOK_frame1.png | Técnica: GradCAM | Entropia: -0.1030 | Ruído: 0.0839 | Geo: 0.2497 | AOS: 6.7418
[2/6999] Imagem: DeepFakeDetection_01_02__meeting_serious__YVGY8LOK_frame2.png | Técnica: Saliency | Entropia: -0.0635 | Ruído

[10/6999] Imagem: DeepFakeDetection_01_03__talking_against_wall__JZUXXFRB_frame5.png | Técnica: GradCAM | Entropia: -0.0991 | Ruído: 0.1322 | Geo: 0.3013 | AOS: 5.5554
[11/6999] Imagem: DeepFakeDetection_01_04__meeting_serious__0XUW13RW_frame1.png | Técnica: Saliency | Entropia: -0.0585 | Ruído: 0.3691 | Geo: 0.2672 | AOS: 7.8145
[11/6999] Imagem: DeepFakeDetection_01_04__meeting_serious__0XUW13RW_frame1.png | Técnica: InputXGradient | Entropia: -0.0629 | Ruído: 0.3332 | Geo: 0.2692 | AOS: 4.5128
[11/6999] Imagem: DeepFakeDetection_01_04__meeting_serious__0XUW13RW_frame1.png | Técnica: IntegratedGradients | Entropia: -0.0588 | Ruído: 0.3537 | Geo: 0.2730 | AOS: 4.0813
[11/6999] Imagem: DeepFakeDetection_01_04__meeting_serious__0XUW13RW_frame1.png | Técnica: SmoothGrad | Entropia: -0.0618 | Ruído: 0.3750 | Geo: 0.2817 | AOS: 4.6185
[11/6999] Imagem: DeepFakeDetection_01_04__meeting_serious__0XUW13RW_frame1.png | Técnica: GradCAM | Entropia: -0.1015 | Ruído: 0.1406 | Geo: 0.3166 | AOS: 6

[20/6999] Imagem: DeepFakeDetection_01_09__talking_angry_couch__P70H1JB0_frame5.png | Técnica: SmoothGrad | Entropia: -0.0621 | Ruído: 0.2807 | Geo: 0.1791 | AOS: 4.5112
[20/6999] Imagem: DeepFakeDetection_01_09__talking_angry_couch__P70H1JB0_frame5.png | Técnica: GradCAM | Entropia: -0.1029 | Ruído: 0.1256 | Geo: 0.3408 | AOS: 5.9907
[21/6999] Imagem: DeepFakeDetection_01_11__meeting_serious__FCNL2OVP_frame1.png | Técnica: Saliency | Entropia: -0.0613 | Ruído: 0.4433 | Geo: 0.3412 | AOS: 3.9703
[21/6999] Imagem: DeepFakeDetection_01_11__meeting_serious__FCNL2OVP_frame1.png | Técnica: InputXGradient | Entropia: -0.0529 | Ruído: 0.3874 | Geo: 0.3528 | AOS: 4.1312
[21/6999] Imagem: DeepFakeDetection_01_11__meeting_serious__FCNL2OVP_frame1.png | Técnica: IntegratedGradients | Entropia: -0.0589 | Ruído: 0.3435 | Geo: 0.3457 | AOS: 3.7944
[21/6999] Imagem: DeepFakeDetection_01_11__meeting_serious__FCNL2OVP_frame1.png | Técnica: SmoothGrad | Entropia: -0.0603 | Ruído: 0.3702 | Geo: 0.3452 | 

[30/6999] Imagem: DeepFakeDetection_01_11__outside_talking_pan_laughing__WIZ7GKOD_frame5.png | Técnica: InputXGradient | Entropia: -0.0569 | Ruído: 0.3756 | Geo: 0.4522 | AOS: 4.6802
[30/6999] Imagem: DeepFakeDetection_01_11__outside_talking_pan_laughing__WIZ7GKOD_frame5.png | Técnica: IntegratedGradients | Entropia: -0.0618 | Ruído: 0.3938 | Geo: 0.4449 | AOS: 4.9827
[30/6999] Imagem: DeepFakeDetection_01_11__outside_talking_pan_laughing__WIZ7GKOD_frame5.png | Técnica: SmoothGrad | Entropia: -0.0636 | Ruído: 0.3613 | Geo: 0.4311 | AOS: 3.2138
[30/6999] Imagem: DeepFakeDetection_01_11__outside_talking_pan_laughing__WIZ7GKOD_frame5.png | Técnica: GradCAM | Entropia: -0.1007 | Ruído: 0.3724 | Geo: 0.2763 | AOS: 3.8651
[31/6999] Imagem: DeepFakeDetection_01_12__outside_talking_still_laughing__TNI7KUZ6_frame1.png | Técnica: Saliency | Entropia: -0.0652 | Ruído: 0.3638 | Geo: 0.3784 | AOS: 8.3650
[31/6999] Imagem: DeepFakeDetection_01_12__outside_talking_still_laughing__TNI7KUZ6_frame1.png 

[39/6999] Imagem: DeepFakeDetection_01_15__outside_talking_pan_laughing__02HILKYO_frame4.png | Técnica: IntegratedGradients | Entropia: -0.0568 | Ruído: 0.2784 | Geo: 0.1673 | AOS: 6.0196
[39/6999] Imagem: DeepFakeDetection_01_15__outside_talking_pan_laughing__02HILKYO_frame4.png | Técnica: SmoothGrad | Entropia: -0.0651 | Ruído: 0.2750 | Geo: 0.1762 | AOS: 3.9239
[39/6999] Imagem: DeepFakeDetection_01_15__outside_talking_pan_laughing__02HILKYO_frame4.png | Técnica: GradCAM | Entropia: -0.1031 | Ruído: 0.3567 | Geo: 0.4833 | AOS: 3.8582
[40/6999] Imagem: DeepFakeDetection_01_15__outside_talking_pan_laughing__02HILKYO_frame5.png | Técnica: Saliency | Entropia: -0.0631 | Ruído: 0.3224 | Geo: 0.1904 | AOS: 4.6510
[40/6999] Imagem: DeepFakeDetection_01_15__outside_talking_pan_laughing__02HILKYO_frame5.png | Técnica: InputXGradient | Entropia: -0.0527 | Ruído: 0.3651 | Geo: 0.1851 | AOS: 6.4295
[40/6999] Imagem: DeepFakeDetection_01_15__outside_talking_pan_laughing__02HILKYO_frame5.png | Té

...

[576/6999] Imagem: DeepFakeDetection_15_06__exit_phone_room__E98QYYXO_frame1.png | Técnica: IntegratedGradients | Entropia: -0.0602 | Ruído: 0.3734 | Geo: 0.1692 | AOS: 5.0210
[576/6999] Imagem: DeepFakeDetection_15_06__exit_phone_room__E98QYYXO_frame1.png | Técnica: SmoothGrad | Entropia: -0.0639 | Ruído: 0.3446 | Geo: 0.1807 | AOS: 4.4794
[576/6999] Imagem: DeepFakeDetection_15_06__exit_phone_room__E98QYYXO_frame1.png | Técnica: GradCAM | Entropia: -0.1024 | Ruído: 0.1405 | Geo: 0.3669 | AOS: 5.0777
[577/6999] Imagem: DeepFakeDetection_15_06__exit_phone_room__E98QYYXO_frame2.png | Técnica: Saliency | Entropia: -0.0634 | Ruído: 0.4630 | Geo: 0.3274 | AOS: 6.5981
[577/6999] Imagem: DeepFakeDetection_15_06__exit_phone_room__E98QYYXO_frame2.png | Técnica: InputXGradient | Entropia: -0.0649 | Ruído: 0.4891 | Geo: 0.3257 | AOS: 4.7227
[577/6999] Imagem: DeepFakeDetection_15_06__exit_phone_room__E98QYYXO_frame2.png | Técnica: IntegratedGradients | Entropia: -0.0669 | Ruído: 0.4126 | Geo: 0.

[585/6999] Imagem: DeepFakeDetection_15_07__walking_down_indoor_hall_disgust__NYPO7U50_frame5.png | Técnica: GradCAM | Entropia: -0.1039 | Ruído: 0.2289 | Geo: 0.4550 | AOS: 3.3196
[586/6999] Imagem: DeepFakeDetection_15_12__talking_angry_couch__N0SRODQD_frame1.png | Técnica: Saliency | Entropia: -0.0622 | Ruído: 0.2042 | Geo: 0.1929 | AOS: 7.3825
[586/6999] Imagem: DeepFakeDetection_15_12__talking_angry_couch__N0SRODQD_frame1.png | Técnica: InputXGradient | Entropia: -0.0553 | Ruído: 0.2044 | Geo: 0.1972 | AOS: 6.1380
[586/6999] Imagem: DeepFakeDetection_15_12__talking_angry_couch__N0SRODQD_frame1.png | Técnica: IntegratedGradients | Entropia: -0.0674 | Ruído: 0.2170 | Geo: 0.1940 | AOS: 5.0369
[586/6999] Imagem: DeepFakeDetection_15_12__talking_angry_couch__N0SRODQD_frame1.png | Técnica: SmoothGrad | Entropia: -0.0593 | Ruído: 0.2200 | Geo: 0.2125 | AOS: 3.0392
[586/6999] Imagem: DeepFakeDetection_15_12__talking_angry_couch__N0SRODQD_frame1.png | Técnica: GradCAM | Entropia: -0.1028 

[595/6999] Imagem: DeepFakeDetection_15_20__kitchen_pan__99NHTFBM_frame5.png | Técnica: SmoothGrad | Entropia: -0.0651 | Ruído: 0.3640 | Geo: 0.4217 | AOS: 3.3768
[595/6999] Imagem: DeepFakeDetection_15_20__kitchen_pan__99NHTFBM_frame5.png | Técnica: GradCAM | Entropia: -0.1013 | Ruído: 0.2105 | Geo: 0.2075 | AOS: 5.7730
[596/6999] Imagem: DeepFakeDetection_17_08__exit_phone_room__CFIY1YEQ_frame1.png | Técnica: Saliency | Entropia: -0.0667 | Ruído: 0.4910 | Geo: 0.3484 | AOS: 7.8091
[596/6999] Imagem: DeepFakeDetection_17_08__exit_phone_room__CFIY1YEQ_frame1.png | Técnica: InputXGradient | Entropia: -0.0592 | Ruído: 0.4074 | Geo: 0.3611 | AOS: 5.2673
[596/6999] Imagem: DeepFakeDetection_17_08__exit_phone_room__CFIY1YEQ_frame1.png | Técnica: IntegratedGradients | Entropia: -0.0603 | Ruído: 0.4226 | Geo: 0.3590 | AOS: 4.3644
[596/6999] Imagem: DeepFakeDetection_17_08__exit_phone_room__CFIY1YEQ_frame1.png | Técnica: SmoothGrad | Entropia: -0.0622 | Ruído: 0.4110 | Geo: 0.3542 | AOS: 3.408

[605/6999] Imagem: DeepFakeDetection_17_16__walk_down_hall_angry__EWJJTZHY_frame5.png | Técnica: IntegratedGradients | Entropia: -0.0592 | Ruído: 0.3203 | Geo: 0.0801 | AOS: 6.5133
[605/6999] Imagem: DeepFakeDetection_17_16__walk_down_hall_angry__EWJJTZHY_frame5.png | Técnica: SmoothGrad | Entropia: -0.0613 | Ruído: 0.3429 | Geo: 0.0919 | AOS: 3.5969
[605/6999] Imagem: DeepFakeDetection_17_16__walk_down_hall_angry__EWJJTZHY_frame5.png | Técnica: GradCAM | Entropia: -0.0965 | Ruído: 0.1188 | Geo: 0.3603 | AOS: 4.1561
[606/6999] Imagem: DeepFakeDetection_17_16__walk_down_hall_angry__P141P1QH_frame1.png | Técnica: Saliency | Entropia: -0.0691 | Ruído: 0.2114 | Geo: 0.3583 | AOS: 4.5452
[606/6999] Imagem: DeepFakeDetection_17_16__walk_down_hall_angry__P141P1QH_frame1.png | Técnica: InputXGradient | Entropia: -0.0656 | Ruído: 0.2327 | Geo: 0.3665 | AOS: 7.4359
[606/6999] Imagem: DeepFakeDetection_17_16__walk_down_hall_angry__P141P1QH_frame1.png | Técnica: IntegratedGradients | Entropia: -0.

KeyboardInterrupt: 

In [ ]:
# COM GPU 

import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import numpy as np
from skimage.segmentation import slic
from scipy.stats import entropy
from skimage.transform import resize, rotate
from skimage.util import random_noise
from captum.attr import IntegratedGradients, Saliency, InputXGradient, NoiseTunnel
import pandas as pd
import gc
from matplotlib import cm
from tqdm import tqdm

# =====================
# CONFIGURAÇÕES
# =====================
MODEL_PATH = "resnet50_multiclass.pth"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DATASET_ROOT = "./face_extraction/faceforensis_face_extract/test/"
class_names = ["DeepFakeDetection", "FaceShifter", "FaceSwap", "NeuraTextures", "deepfakes", "face2face", "original"]
explic_methods = ["Saliency", "InputXGradient", "IntegratedGradients", "SmoothGrad", "GradCAM"]

# =====================
# CARREGA MODELO
# =====================
print(f"Usando dispositivo: {DEVICE}")
model = models.resnet50(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 7)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model = model.to(DEVICE)
model.eval()

# =====================
# FUNÇÕES AUXILIARES
# =====================
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path).convert("RGB")
    tensor = transform(image).unsqueeze(0).to(DEVICE)
    np_img = np.array(image.resize((224, 224))) / 255.0
    return tensor, np_img

def make_gradcam(model, img_tensor, target_layer='layer4.2.conv3'):
    gradients, activations = [], []
    def backward_hook(module, grad_input, grad_output):
        gradients.append(grad_output[0])
    def forward_hook(module, input, output):
        activations.append(output)
    target_module = dict(model.named_modules())[target_layer]
    fh = target_module.register_forward_hook(forward_hook)
    bh = target_module.register_full_backward_hook(backward_hook)
    img_tensor.requires_grad_(True)
    output = model(img_tensor)
    class_idx = torch.argmax(output)
    loss = output[0, class_idx]
    model.zero_grad()
    loss.backward()
    grad = gradients[0]
    act = activations[0]
    pooled_grad = torch.mean(grad, dim=[0, 2, 3])
    for i in range(act.shape[1]):
        act[:, i, :, :] *= pooled_grad[i]
    heatmap = act.mean(dim=1).squeeze().detach().cpu().numpy()
    heatmap = np.maximum(heatmap, 0)
    heatmap = heatmap / (heatmap.max() + 1e-12)
    fh.remove()
    bh.remove()
    return heatmap

def get_saliency_map(method, model, input_tensor, target_idx):
    input_tensor = input_tensor.clone().detach().requires_grad_(True).to(DEVICE)
    if method == "Saliency":
        explainer = Saliency(model)
        attributions = explainer.attribute(input_tensor, target=target_idx)
    elif method == "InputXGradient":
        explainer = InputXGradient(model)
        attributions = explainer.attribute(input_tensor, target=target_idx)
    elif method == "IntegratedGradients":
        explainer = IntegratedGradients(model)
        attributions = explainer.attribute(input_tensor, target=target_idx, n_steps=10, internal_batch_size=5)
    elif method == "SmoothGrad":
        saliency = Saliency(model)
        explainer = NoiseTunnel(saliency)
        attributions = explainer.attribute(input_tensor, nt_type='smoothgrad', nt_samples=15, target=target_idx)
    elif method == "GradCAM":
        cam = make_gradcam(model, input_tensor)
        return resize(cam, (224, 224), mode='reflect', anti_aliasing=True)
    else:
        raise ValueError(f"Método {method} desconhecido.")
    attr_np = attributions.detach().cpu().numpy().squeeze()
    attr_np = np.abs(attr_np).sum(axis=0)
    return (attr_np - attr_np.min()) / (attr_np.max() - attr_np.min() + 1e-12)

# =====================
# MÉTRICAS
# =====================

def safe_corrcoef(a, b):
    """Calcula correlação de Pearson, retornando 0 se algum vetor for constante."""
    if np.std(a) == 0 or np.std(b) == 0:
        return 0.0
    return np.corrcoef(a, b)[0, 1]



def AOS(model, image_np, saliency_map, ks=[1, 2, 3]):
    segs = slic((image_np * 255).astype(np.uint8), n_segments=100, compactness=20, start_label=0)
    relevance = np.array([np.mean(saliency_map[segs == i]) for i in range(np.max(segs) + 1)])
    order = np.argsort(relevance)[::-1]
    def model_acc(img_np):
        tensor = torch.from_numpy(img_np.transpose(2, 0, 1)).float().unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            out = model(tensor)
            prob = torch.softmax(out, dim=1).max().item()
        return prob
    acc_orig = model_acc(image_np)
    aos_values = []
    for k in ks:
        mask = np.isin(segs, order[:k]).astype(np.float32)
        mask = np.repeat(mask[:, :, np.newaxis], 3, axis=2)
        perturbed = image_np.copy()
        perturbed[mask == 1] = random_noise(perturbed[mask == 1], mode='gaussian', var=0.05)
        aos_values.append(acc_orig - model_acc(perturbed))
    return aos_values

def S_entropy(smap):
    p = smap / (np.sum(smap) + 1e-12)
    return -np.sum(p * np.log(p + 1e-12))

def S_noise(model, image_np, method):
    tensor = torch.from_numpy(image_np.transpose(2, 0, 1)).float().unsqueeze(0).to(DEVICE)
    pred_idx = torch.argmax(model(tensor)).item()
    S_clean = get_saliency_map(method, model, tensor, pred_idx)
    noisy_img = np.clip(image_np + np.random.normal(0, 0.1, image_np.shape), 0, 1)
    tensor_noisy = torch.from_numpy(noisy_img.transpose(2, 0, 1)).float().unsqueeze(0).to(DEVICE)
    S_noisy = get_saliency_map(method, model, tensor_noisy, pred_idx)
    return safe_corrcoef(S_clean.flatten(), S_noisy.flatten())

def S_resilience(model, image_np, method):
    def apply_transforms(img):
        return [
            np.flip(img, axis=1).copy(),  # flip horizontal com copy
            rotate(img, angle=15, mode='reflect').copy(),  # rotação com copy
            np.roll(img, shift=10, axis=1).copy()  # translação com copy
        ]
    tensor = torch.from_numpy(image_np.transpose(2, 0, 1)).float().unsqueeze(0).to(DEVICE)
    pred_idx = torch.argmax(model(tensor)).item()
    S_orig = get_saliency_map(method, model, tensor, pred_idx)
    sims = []
    for t_img in apply_transforms(image_np):
        t_tensor = torch.from_numpy(t_img.transpose(2, 0, 1)).float().unsqueeze(0).to(DEVICE)
        S_t = get_saliency_map(method, model, t_tensor, pred_idx)
        #sims.append(np.corrcoef(S_orig.flatten(), S_t.flatten())[0, 1])
        sims.append(safe_corrcoef(S_orig.flatten(), S_t.flatten()))

    return np.mean(sims)


# =====================
# EXECUÇÃO PRINCIPAL
# =====================
image_files = []
image_labels = []
for folder in class_names:
    folder_path = os.path.join(DATASET_ROOT, folder)
    files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.png'))])
    for f in files:
        image_files.append(os.path.join(folder_path, f))
        image_labels.append(folder)

resultados = []
for path, label in tqdm(zip(image_files, image_labels), total=len(image_files), desc="Processando imagens"):
    img_tensor, raw_img = preprocess_image(path)
    output = model(img_tensor)
    pred_idx = torch.argmax(output, dim=1).item()
    for method in explic_methods:
        salmap = get_saliency_map(method, model, img_tensor, pred_idx)
        aos_1, aos_2, aos_3 = AOS(model, raw_img, salmap)
        sentropy_val = S_entropy(salmap)
        snoise_val = S_noise(model, raw_img, method)
        sres_val = S_resilience(model, raw_img, method)
        resultados.append({
            "imagem": os.path.basename(path),
            "classe_real": label,
            "classe_predita": class_names[pred_idx],
            "metodo_explicabilidade": method,
            "AOS_1": aos_1,
            "AOS_2": aos_2,
            "AOS_3": aos_3,
            "S_entropy": sentropy_val,
            "S_noise": snoise_val,
            "S_resilience": sres_val
        })
    del img_tensor, raw_img, output
    torch.cuda.empty_cache()
    gc.collect()

df_resultados = pd.DataFrame(resultados)
df_resultados.to_excel("resultados_metricas_tsigos_crum.xlsx", index=False)
print("✅ Planilha salva como resultados_metricas_tsigos_crum.xlsx")


Usando dispositivo: cuda


Processando imagens: 100%|█████████▉| 6974/6999 [5:32:40<01:10,  2.83s/it]  